In [1]:
from costly.costlog import Costlog
from time import sleep

costlog = Costlog()

with costlog.new_item() as (item, timer):
    item["cost"] = 10
    sleep(1.5)
    item["time"] = timer()

In [7]:
Costlog.CostlogItem().__annotations__

{'cost_min': float, 'cost_max': float, 'time_min': float, 'time_max': float}